In [51]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import Adam

import numpy as np
import os

LEARNING_RATE = 0.0005
BATCH_SIZE = 128
EPOCHS = 12

In [52]:
import sys
sys.path.insert(1, 'D:\Code\Data Science\Projects\music_generador_deep_learning\src/modeling')

from vq_vae import VQ_VAE
from train_vq import *

In [53]:
import sys
sys.path.insert(1, 'D:\Code\Data Science\Projects\music_generador_deep_learning\src')

from generate import *

In [54]:
SPECTROGRAMS_PATH = "../data/fsdd/spectrograms"

LEARNING_RATE = 0.0005
BATCH_SIZE = 64
EPOCHS = 30

In [55]:
x_train, _ = load_fsdd(SPECTROGRAMS_PATH)
print(len(x_train))
print(len(x_train[0]))

3000
256


In [56]:
data_variance = np.var(x_train / 255.0)

In [57]:
VQVAE = VQ_VAE(
    input_shape=(256, 64, 1),
    conv_filters=(256, 128, 64),
    conv_kernels=(3, 3, 3),
    conv_strides=(2, 2, 2),
    latent_space_dim=16,
    data_variance=data_variance,
    embeddings_size=128
)
VQVAE.summary()


Model: "encoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ encoder_input (InputLayer)      │ (None, 256, 64, 1)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_layer_0 (Conv2D)           │ (None, 128, 32, 256)   │         2,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_relu_0 (ReLU)           │ (None, 128, 32, 256)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_bn_0                    │ (None, 128, 32, 256)   │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_layer_1 (Conv2D)           │ (None, 64, 16, 128)    │       295,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_relu_1 (ReLU)           │ (None, 64, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_bn_1                    │ (None, 64, 16, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_layer_2 (Conv2D)           │ (None, 32, 8, 64)      │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_relu_2 (ReLU)           │ (None, 32, 8, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder_bn_2                    │ (None, 32, 8, 64)      │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 32, 8, 16)      │         1,040 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 374,224 (1.43 MB)

 Trainable params: 373,328 (1.42 MB)

 Non-trainable params: 896 (3.50 KB)

...

Model: "variational_autoencoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ encoder_input (InputLayer)      │ (None, 256, 64, 1)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder (Functional)            │ (None, 32, 8, 16)      │       374,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vector_quantizer_4              │ (None, 32, 8, 16)      │             0 │
│ (VectorQuantizer)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder (Functional)            │ (None, 256, 64, 1)     │        85,057 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 459,281 (1.75 MB)

 Trainable params: 458,001 (1.75 MB)

 Non-trainable params: 1,280 (5.00 KB)

In [58]:
# VQVAE = VQ_VAE(
#     input_shape=(256, 64, 1),
#     conv_filters=(512, 256, 128, 64, 32),
#     conv_kernels=(3, 3, 3, 3, 3),
#     conv_strides=(2, 2, 2, 2, (2, 1)),
#     data_variance=data_variance,
#     embeddings_size=128,
#     latent_space_dim=128
# )
# VQVAE.summary()


In [59]:
VQVAE.compile(LEARNING_RATE)

In [60]:
VQVAE.train(x_train, BATCH_SIZE, EPOCHS)

Epoch 1/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 23s 445ms/step - reconstruction_loss: 283473.9688 - total_loss: 283627.6875 - vq_loss: 139.3331
Epoch 2/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 21s 440ms/step - reconstruction_loss: 75532.6719 - total_loss: 339461.5000 - vq_loss: 263927.5938
Epoch 3/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 21s 440ms/step - reconstruction_loss: 52940.8203 - total_loss: 41938172.0000 - vq_loss: 41885232.0000
Epoch 4/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 21s 438ms/step - reconstruction_loss: 32832.6250 - total_loss: 1971223296.0000 - vq_loss: 1971190528.0000
Epoch 5/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 21s 437ms/step - reconstruction_loss: 18166.5020 - total_loss: 21427603456.0000 - vq_loss: 21427585024.0000
Epoch 6/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 23s 487ms/step - reconstruction_loss: 13447.1885 - total_loss: 45705748480.0000 - vq_loss: 45705740288.0000
Epoch 7/30
47/47 ━━━━━━━━━━━━━━━━━━━━ 23s 479ms/step - reconstruction_loss: 12301.2012 - total_loss: 46467436544.0000 - vq_loss: 46467424256.0000
Epoch 8/

In [61]:
VQVAE.save("../model/vq_vae")

Model saved successfully in folder: ../model/vq_vae


In [62]:
# VQVAE = VQ_VAE.load("model")
sound_generator = SoundGenerator(VQVAE, HOP_LENGTH)


In [63]:
MIN_MAX_VALUES_PATH = "../data/fsdd/min_max_values.pkl"
SAVE_DIR_ORIGINAL = "samples/vq_vae/original/"
SAVE_DIR_GENERATED = "samples/vq_vae/generated/"


In [64]:
# Load spectrograms + min max values
with open(MIN_MAX_VALUES_PATH, "rb") as f:
    min_max_values = pickle.load(f)
specs, file_paths = load_fsdd(SPECTROGRAMS_PATH)


In [65]:
# Sample spectrograms + min max values

file_paths_selected = file_paths

sampled_indexes = np.random.choice(range(len(specs)), 5)
sampled_spectrogrmas = specs[sampled_indexes]

file_paths_selected = [file_paths_selected[index] for index in sampled_indexes]
file_paths_selected =  list(map(lambda st: str.replace(st, "\\", "/"), file_paths_selected))
file_paths_selected =  list(map(lambda st: str.replace(st, "..", "."), file_paths_selected))

sampled_min_max_values = [min_max_values[file_path] for file_path in file_paths_selected]

print(file_paths_selected)
print(sampled_min_max_values)



['./data/fsdd/spectrograms/0_george_33.wav.npy', './data/fsdd/spectrograms/8_theo_1.wav.npy', './data/fsdd/spectrograms/0_theo_27.wav.npy', './data/fsdd/spectrograms/9_yweweler_21.wav.npy', './data/fsdd/spectrograms/8_george_17.wav.npy']
[{'min': -52.57717, 'max': 27.422827}, {'min': -79.73695, 'max': 0.26304722}, {'min': -83.88965, 'max': -3.8896465}, {'min': -75.14155, 'max': 4.8584523}, {'min': -52.92492, 'max': 27.07508}]


In [66]:
# Generate audio for sampled spectrograms
signals, _ = sound_generator.generate(sampled_spectrogrmas, sampled_min_max_values)

original_signals = sound_generator.convert_spectrograms_to_audio(sampled_spectrogrmas, sampled_min_max_values)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step


In [67]:
save_signals(signals, SAVE_DIR_GENERATED)
save_signals(original_signals, SAVE_DIR_ORIGINAL)